In [82]:
import pandas as pd
from sqlalchemy import create_engine

db_user = 'root'
db_password = 'mySQL12345*'
db_host = 'localhost'
db_name = 'nfl_statistics'

# Create the database engine
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

In [83]:
# Load Basic_Stats.csv into a data-frame
basic_stats_df = pd.read_csv('Basic_Stats.csv')

# Load Career Statistic files into dataframes
passing_df = pd.read_csv('Career_Stats_Passing.csv')
receiving_df = pd.read_csv('Career_Stats_Receiving.csv')
rushing_df = pd.read_csv('Career_Stats_Rushing.csv')
defensive_df = pd.read_csv('Career_Stats_Defensive.csv')

In [84]:
# Extract unique high schools
unique_high_schools_df = basic_stats_df[['High School', 'High School Location']].dropna().drop_duplicates()
unique_high_schools_df.columns = ['Name', 'Location']  # Rename columns to match schema

# Load unique high schools dataset into High_School table
unique_high_schools_df.to_sql('high_school', con=engine, if_exists='append', index=False)

print('High-school data has been inserted.')

High-school data has been inserted.


In [85]:
# Extract unique colleges
unique_colleges_df = basic_stats_df[['College']].dropna().drop_duplicates()
unique_colleges_df.columns = ['Name']  # Rename column to match schema

# Load unique colleges dataset into College table
unique_colleges_df.to_sql('college', con=engine, if_exists='append', index=False)

print('College data has been inserted.')

College data has been inserted.


In [86]:
# Extract unique teams from each Career Statistic file
unique_teams_receiving_df = passing_df[['Team']].drop_duplicates()
unique_teams_receiving_df = receiving_df[['Team']].drop_duplicates()
unique_teams_rushing_df = rushing_df[['Team']].drop_duplicates()
unique_teams_defensive_df = defensive_df[['Team']].drop_duplicates()

# Extract unique teams from Basic Stats file
unique_teams_basic_stats_df = basic_stats_df[['Current Team']].drop_duplicates()
unique_teams_basic_stats_df.columns = ['Team']  # Rename column to match schema

# Concatenate unique teams from all sources
unique_teams_df = pd.concat([unique_teams_receiving_df, unique_teams_receiving_df, 
                             unique_teams_rushing_df, unique_teams_defensive_df,
                             unique_teams_basic_stats_df], ignore_index=True)

# Drop any null values and duplicates
unique_teams_df.dropna(inplace=True)
unique_teams_df.drop_duplicates(inplace=True)

# Rename column 'Team' to 'TeamName'
unique_teams_df.rename(columns={'Team': 'TeamName'}, inplace=True)

# Load unique teams dataset into the Teams table
unique_teams_df.to_sql('team', con=engine, if_exists='append', index=False)

print('Team data has been inserted.')

Team data has been inserted.


In [87]:
# Standardize player names
basic_stats_df['PlayerName'] = basic_stats_df['Name'].apply(lambda x: ' '.join(reversed(x.split(', '))))

# Match team, college, and high school names with corresponding unique IDs
team_mapping_df = pd.read_sql_table('team', engine)
college_mapping_df = pd.read_sql_table('college', engine)
high_school_mapping_df = pd.read_sql_table('high_school', engine)

# Rename the 'Name' column to 'CollegeName' in college_mapping_df
college_mapping_df.rename(columns={'Name': 'CollegeName'}, inplace=True)

# Rename the 'Name' column to 'HSchoolName' in high_school_mapping_df
high_school_mapping_df.rename(columns={'Name': 'HSchoolName'}, inplace=True)

basic_stats_df = basic_stats_df.merge(team_mapping_df, left_on='Current Team', right_on='TeamName', how='left')
basic_stats_df = basic_stats_df.merge(college_mapping_df, left_on='College', right_on='CollegeName', how='left')
basic_stats_df = basic_stats_df.merge(high_school_mapping_df, left_on='High School', right_on='HSchoolName', how='left')

# Select relevant columns
player_df = basic_stats_df[['Player Id', 'PlayerName', 'Number', 'Years Played', 'Position', 'Current Status',
                            'Height (inches)', 'Weight (lbs)', 'Age', 'Birthday', 'Birth Place', 'Experience',
                            'TeamID', 'CollegeID', 'HighSchoolID']]

# Rename columns to match Player table schema
player_df.rename(columns={'Player Id': 'PlayerID',
                          'Years Played': 'YearsPlayed',
                          'Current Status': 'CurrentStatus',
                          'Height (inches)': 'Height',
                          'Weight (lbs)': 'Weight',
                          'Birth Place': 'BirthPlace',
                          'TeamID': 'CurrentTeamID'}, inplace=True)

# Convert 'Birthday' column to datetime format with error handling
player_df['Birthday'] = pd.to_datetime(player_df['Birthday'], format='%m/%d/%Y', errors='coerce')

# Convert datetime format to 'YYYY-MM-DD' format
player_df['Birthday'] = player_df['Birthday'].dt.strftime('%Y-%m-%d')

# Remove duplicate records based on PlayerID
player_df.drop_duplicates(subset=['PlayerID'], inplace=True)

# Load processed player data into the Player table
player_df.to_sql('player', con=engine, if_exists='append', index=False)

print('Player data has been inserted.')

C:\Users\sreej\AppData\Local\Temp\ipykernel_17476\4060616704.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df.rename(columns={'Player Id': 'PlayerID',
C:\Users\sreej\AppData\Local\Temp\ipykernel_17476\4060616704.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df['Birthday'] = pd.to_datetime(player_df['Birthday'], format='%m/%d/%Y', errors='coerce')
C:\Users\sreej\AppData\Local\Temp\ipykernel_17476\4060616704.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

Player data has been inserted.


In [88]:
# Add 'StatsType' column to each data-frame
passing_df['StatsType'] = 'Passing'
receiving_df['StatsType'] = 'Receiving'
rushing_df['StatsType'] = 'Rushing'
defensive_df['StatsType'] = 'Defensive'

# Extract unique combinations of PlayerID, Team, Year, and StatsType
unique_stats = pd.concat([passing_df[['Player Id', 'Team', 'Year', 'StatsType']],
                          receiving_df[['Player Id', 'Team', 'Year', 'StatsType']],
                          rushing_df[['Player Id', 'Team', 'Year', 'StatsType']],
                          defensive_df[['Player Id', 'Team', 'Year', 'StatsType']]])

unique_stats.drop_duplicates(inplace=True)

# Map Team column to TeamName column to get TeamID
unique_stats = unique_stats.merge(team_mapping_df, left_on='Team', right_on='TeamName', how='left')
unique_stats.drop(columns=['Team', 'TeamName'], inplace=True)  # Drop unnecessary columns

# Rename the 'Player Id' column to 'PlayerID' in unique_stats data-frame
unique_stats.rename(columns={'Player Id': 'PlayerID'}, inplace=True)

# Load unique combinations into the Yearly_Statistics table
unique_stats.to_sql('yearly_statistics', con=engine, if_exists='append', index=False)

print('Yearly statistics data has been inserted.')

Yearly statistics data has been inserted.


In [89]:
# Replace '--' with blanks in each career statistics dataframe
receiving_df.replace('--', pd.NA, inplace=True)
receiving_df.replace('--', pd.NA, inplace=True)
rushing_df.replace('--', pd.NA, inplace=True)
defensive_df.replace('--', pd.NA, inplace=True)

# Rename the 'Player Id' column to 'PlayerID' in the data-frames
passing_df.rename(columns={'Player Id': 'PlayerID'}, inplace=True)
receiving_df.rename(columns={'Player Id': 'PlayerID'}, inplace=True)
rushing_df.rename(columns={'Player Id': 'PlayerID'}, inplace=True)
defensive_df.rename(columns={'Player Id': 'PlayerID'}, inplace=True)

# Load the yearly statistics data to map StatID
yearly_stats_df = pd.read_sql_table('yearly_statistics', engine)

# Merge the CSV data with the team dataframe on TeamName to get TeamID
passing_df = passing_df.merge(team_mapping_df, left_on='Team', right_on='TeamName', how='left')
receiving_df = receiving_df.merge(team_mapping_df, left_on='Team', right_on='TeamName', how='left')
rushing_df = rushing_df.merge(team_mapping_df, left_on='Team', right_on='TeamName', how='left')
defensive_df = defensive_df.merge(team_mapping_df, left_on='Team', right_on='TeamName', how='left')

# Merge with yearly statistics to get StatID
passing_df = passing_df.merge(yearly_stats_df, on=['PlayerID', 'TeamID', 'Year', 'StatsType'], how='left')
receiving_df = receiving_df.merge(yearly_stats_df, on=['PlayerID', 'TeamID', 'Year', 'StatsType'], how='left')
rushing_df = rushing_df.merge(yearly_stats_df, on=['PlayerID', 'TeamID', 'Year', 'StatsType'], how='left')
defensive_df = defensive_df.merge(yearly_stats_df, on=['PlayerID', 'TeamID', 'Year', 'StatsType'], how='left')

# Rename columns to match passing_statistics table schema
passing_df.rename(columns={
    'Passes Attempted':'PassesAttempted',
    'Passes Completed':'PassesCompleted',
    'Completion Percentage':'CompletionPercentage',
    'Pass Attempts Per Game':'PassAttemptsPerGame',
    'Passing Yards':'PassingYards',
    'Passing Yards Per Attempt':'PassingYardsPerAttempt',
    'Passing Yards Per Game':'PassingYardsPerGame',
    'TD Passes':'TDPasses',
    'Percentage of TDs per Attempts':'PercentageTDsPerAttempt',
    'Int Rate':'IntRate',
    'Longest Pass':'LongestPass',
    'Passes Longer than 20 Yards':'PassesLongerThan20Yards',
    'Passes Longer than 40 Yards':'PassesLongerThan40Yards',
    'Sacked Yards Lost':'SackedYardsLost',
    'Passer Rating':'PasserRating'
}, inplace=True)

# Select columns for passing_statistics table
passing_df = passing_df[[
    'StatID',
    'PassesAttempted',
    'PassesCompleted',
    'CompletionPercentage',
    'PassAttemptsPerGame',
    'PassingYards',
    'PassingYardsPerAttempt',
    'PassingYardsPerGame',
    'TDPasses',
    'PercentageTDsPerAttempt',
    'Ints',
    'IntRate',
    'LongestPass',
    'PassesLongerThan20Yards',
    'PassesLongerThan40Yards',
    'Sacks',
    'SackedYardsLost',
    'PasserRating'
]]

# Replace '--' with -1 as a placeholder for NAs.
passing_df.replace('--', -1, inplace=True)

# Convert strings to integers
passing_df['PassingYards'] = passing_df['PassingYards'].apply(lambda x: int(x.replace(',', '')) if isinstance(x, str) else x)

# Replace the placeholder value with NaN
passing_df['PassingYards'].replace(-1, pd.NA, inplace=True)

# Rename columns to match receiving_statistics table schema
receiving_df.rename(columns={
    'Receiving Yards':'ReceivingYards',
    'Yards Per Reception':'YardsPerReception',
    'Yards Per Game':'YardsPerGame',
    'Longest Reception':'LongestReception',
    'Receiving TDs':'ReceivingTDs',
    'Receptions Longer than 20 Yards':'ReceptionsLongerThan20Yards',
    'Receptions Longer than 40 Yards':'ReceptionsLongerThan40Yards',
    'First Down Receptions':'FirstDownReceptions'
}, inplace=True)

# Select columns for receiving_statistics table
receiving_df = receiving_df[[
    'StatID',
    'Receptions',
    'ReceivingYards',
    'YardsPerReception',
    'YardsPerGame',
    'LongestReception',
    'ReceivingTDs',
    'ReceptionsLongerThan20Yards',
    'ReceptionsLongerThan40Yards',
    'FirstDownReceptions',
    'Fumbles'
]]

# Replace '--' with -1 as a placeholder for NAs.
receiving_df.replace('--', -1, inplace=True)

# Convert strings to integers
receiving_df['ReceivingYards'] = receiving_df['ReceivingYards'].apply(lambda x: int(x.replace(',', '')) if isinstance(x, str) else x)

# Replace the placeholder value with NaN
receiving_df['ReceivingYards'].replace(-1, pd.NA, inplace=True)

# Rename columns to match rushing_statistics table schema
rushing_df.rename(columns={
    'Rushing Attempts':'RushingAttempts',
    'Rushing Attempts Per Game':'RushingAttemptsPerGame',
    'Rushing Yards':'RushingYards',
    'Yards Per Carry':'YardsPerCarry',
    'Rushing Yards Per Game':'RushingYardsPerGame',
    'Rushing TDs':'RushingTDs',
    'Longest Rushing Run':'LongestRushingRun',
    'Rushing First Downs':'RushingFirstDowns',
    'Percentage of Rushing First Downs':'PercentageRushingFirstDowns',
    'Rushing More Than 20 Yards':'RushingMoreThan20Yards',
    'Rushing More Than 40 Yards':'RushingMoreThan40Yards',
}, inplace=True)

# Select columns for rushing_statistics table
rushing_df = rushing_df[[
    'StatID',
    'RushingAttempts',
    'RushingAttemptsPerGame',
    'RushingYards',
    'YardsPerCarry',
    'RushingYardsPerGame',
    'RushingTDs',
    'LongestRushingRun',
    'RushingFirstDowns',
    'PercentageRushingFirstDowns',
    'RushingMoreThan20Yards',
    'RushingMoreThan40Yards',
    'Fumbles'
    ]]

# Replace '--' with -1 as a placeholder for NAs.
rushing_df.replace('--', -1, inplace=True)

# Convert strings to integers
rushing_df['RushingYards'] = rushing_df['RushingYards'].apply(lambda x: int(x.replace(',', '')) if isinstance(x, str) else x)

# Replace the placeholder value with NaN
rushing_df['RushingYards'].replace(-1, pd.NA, inplace=True)

# Rename columns to match defensive_statistics table schema
defensive_df.rename(columns={
    'Total Tackles':'TotalTackles',
    'Solo Tackles':'SoloTackles',
    'Assisted Tackles':'AssistedTackles',
    'Passes Defended':'PassesDefended',
    'Ints for TDs':'IntsforTDs',
    'Int Yards':'IntYards',
    'Yards Per Int':'YardsPerInt',
    'Longest Int Return':'LongestIntReturn',
}, inplace=True)

# Select columns for defensive_statistics table
defensive_df = defensive_df[[
    'StatID',
    'TotalTackles',
    'SoloTackles',
    'AssistedTackles',
    'Sacks',
    'Safties',
    'PassesDefended',
    'Ints',
    'IntsforTDs',
    'IntYards',
    'YardsPerInt',
    'LongestIntReturn',
]]

# Insert the data into the PASSING_STATISTICS table
passing_df.to_sql('passing_statistics', engine, if_exists='append', index=False)

print('Passing statistics data has been inserted.')

# Insert the data into the RECEIVING_STATISTICS table
receiving_df.to_sql('receiving_statistics', engine, if_exists='append', index=False)

print('Receiving statistics data has been inserted.')

# Insert the data into the RUSHING_STATISTICS table
rushing_df.to_sql('rushing_statistics', engine, if_exists='append', index=False)

print('Rushing statistics data has been inserted.')

# Insert the data into the DEFENSIVE_STATISTICS table
defensive_df.to_sql('defensive_statistics', engine, if_exists='append', index=False)

print('Defensive statistics data has been inserted.')


Passing statistics data has been inserted.
Receiving statistics data has been inserted.
Rushing statistics data has been inserted.
Defensive statistics data has been inserted.
